In [1]:
from MVPAnalyzer import Analyzer
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

mvpa = Analyzer('C:/Users/maxim/Documents/INGE CNRS/ESSTECH25/MVP300_DATA/Stationary Profiling/',subdirs=True)

# mvpa.help()

mvpa.load_mvp_data() 

# mvpa.load_ctd_data('C:/Users/maxim/Documents/INGE CNRS/ESSTECH25/BATHYSONDE/DATA/TRAIT/CNV/')


c:\Users\maxim\miniforge3\envs\oceanapp\Lib\site-packages\seabird\cnv.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Found 6 MVP files in the directory: C:/Users/maxim/Documents/INGE CNRS/ESSTECH25/MVP300_DATA/Stationary Profiling/
MVP data loaded successfully.


In [2]:
mvpa.print_profile_metadata()

MVP data:
Number of profiles: 6
  Profil down 0 - Profil up 1 - Latitude: 42.89352, Longitude: 6.24231, Date/Heure: 2025-08-09 10:28:58.600000
  Profil down 2 - Profil up 3 - Latitude: 42.78807, Longitude: 6.17795, Date/Heure: 2025-08-11 09:54:09
  Profil down 4 - Profil up 5 - Latitude: 42.70605, Longitude: 6.14948, Date/Heure: 2025-08-11 07:00:47.600000
  Profil down 6 - Profil up 7 - Latitude: 42.70608, Longitude: 6.14825, Date/Heure: 2025-08-11 07:10:43.600000
  Profil down 8 - Profil up 9 - Latitude: 42.47797, Longitude: 6.12631, Date/Heure: 2025-08-11 03:32:04.200000
  Profil down 10 - Profil up 11 - Latitude: 42.30967, Longitude: 6.09565, Date/Heure: 2025-08-11 00:01:16.800000


In [2]:
# mvpa.keep_selected_profiles([0, 10, 8, 2], [0, 2, 4, 8])
mvpa.keep_selected_profiles([0,10,8,2])
# mvpa.keep_selected_profiles([10,8])

export mvp data to netcdf

In [ ]:
from netCDF4 import Dataset
import gsw
from datetime import datetime

# Concatenate MVP profiles 2 and 3 and export to NetCDF

# Select profiles to concatenate (0-based indexing)
profile_ids = [2, 3]

def concat_from_attr(obj, attr, ids):
    if not hasattr(obj, attr):
        return None
    arr = getattr(obj, attr)
    out = []
    for i in ids:
        ai = np.asarray(arr[i])
        out.append(ai.astype(float))
    return np.concatenate(out)

# Base variables (pressure as base dimension)
pres = concat_from_attr(mvpa, 'PRES_mvp', profile_ids)
temp = concat_from_attr(mvpa, 'TEMP_mvp', profile_ids)
cond = concat_from_attr(mvpa, 'COND_mvp', profile_ids)
do   = concat_from_attr(mvpa, 'DO_mvp',   profile_ids)
turb = concat_from_attr(mvpa, 'TURB_mvp', profile_ids) if hasattr(mvpa, 'TURB_mvp') else None
fluo = concat_from_attr(mvpa, 'FLUO_mvp', profile_ids) if hasattr(mvpa, 'FLUO_mvp') else None
ph   = concat_from_attr(mvpa, 'PH_mvp',   profile_ids) if hasattr(mvpa, 'PH_mvp')   else None
suna = concat_from_attr(mvpa, 'SUNA_mvp', profile_ids) if hasattr(mvpa, 'SUNA_mvp') else None

# Optional per-sample metadata (lat, lon, time) if available
def concat_meta(meta_attr, ids, like_len):
    if not hasattr(mvpa, meta_attr):
        return None
    meta = getattr(mvpa, meta_attr)
    out = []
    for i in ids:
        mi = meta[i]
        # Expand scalar per-profile metadata to per-sample if needed
        if np.isscalar(mi) or (np.asarray(mi).ndim == 0):
            # length from pressure for that profile
            n_i = len(np.asarray(getattr(mvpa, 'PRES_mvp')[i]))
            out.append(np.full(n_i, float(mi)))
        else:
            out.append(np.asarray(mi).astype(float))
    cat = np.concatenate(out)
    # If mismatched length, pad/trim to match base length
    if like_len is not None and len(cat) != like_len:
        if len(cat) > like_len:
            cat = cat[:like_len]
        else:
            pad = np.full(like_len - len(cat), np.nan)
            cat = np.concatenate([cat, pad])
    return cat

lat = concat_meta('LAT_mvp', profile_ids, len(pres)) if hasattr(mvpa, 'LAT_mvp') else None
lon = concat_meta('LON_mvp', profile_ids, len(pres)) if hasattr(mvpa, 'LON_mvp') else None

# Try to get time; handle Python datetime, numpy datetime64, or numeric
time_concat = None
if hasattr(mvpa, 'TIME_mvp'):
    # best-effort conversion to seconds since epoch
    tl = []
    for i in profile_ids:
        ti = getattr(mvpa, 'TIME_mvp')[i]
        ti = np.asarray(ti)
        if np.issubdtype(ti.dtype, np.datetime64):
            ti_sec = ti.astype('datetime64[ns]').astype('int64') / 1e9
        elif np.issubdtype(ti.dtype, np.number):
            ti_sec = ti.astype(float)  # assume already seconds
        else:
            # try parsing strings to timestamps
            ti_sec = np.array([np.datetime64(t).astype('datetime64[ns]').astype('int64')/1e9 for t in ti])
        tl.append(ti_sec)
    time_concat = np.concatenate(tl)
    # Pad/trim
    if len(time_concat) != len(pres):
        if len(time_concat) > len(pres):
            time_concat = time_concat[:len(pres)]
        else:
            time_concat = np.concatenate([time_concat, np.full(len(pres)-len(time_concat), np.nan)])

# Compute Practical Salinity (unitless) where C, T, P are valid
sal = np.full(len(pres), np.nan)
if (cond is not None) and (temp is not None) and (pres is not None):
    mask = (~np.isnan(cond)) & (~np.isnan(temp)) & (~np.isnan(pres))
    if np.any(mask):
        sal_vals = gsw.SP_from_C(cond[mask], temp[mask], pres[mask])
        sal[mask] = sal_vals

# Create NetCDF file
outfile = 'mvp_profiles_2_3.nc'
nc = Dataset(outfile, 'w', format='NETCDF4')

# Dimensions
nc.createDimension('obs', len(pres))

# Variables
v_pres = nc.createVariable('pres', 'f4', ('obs',), zlib=True, fill_value=np.nan)
v_temp = nc.createVariable('temp', 'f4', ('obs',), zlib=True, fill_value=np.nan)
v_sal  = nc.createVariable('sal',  'f4', ('obs',), zlib=True, fill_value=np.nan)
v_do   = nc.createVariable('oxygen_percent', 'f4', ('obs',), zlib=True, fill_value=np.nan)

v_pres.units = 'dbar'
v_pres.long_name = 'Sea water pressure'
v_pres.standard_name = 'sea_water_pressure'

v_temp.units = 'degree_Celsius'
v_temp.long_name = 'Sea water in-situ temperature'
v_temp.standard_name = 'sea_water_temperature'

v_sal.units = '1'
v_sal.long_name = 'Practical Salinity (PSS-78)'
v_sal.standard_name = 'sea_water_practical_salinity'

v_do.units = 'percent'
v_do.long_name = 'Dissolved oxygen saturation (%)'

# Optional variable
if turb is not None:
    v_turb = nc.createVariable('turbidity', 'f4', ('obs',), zlib=True, fill_value=np.nan)
    v_turb.units = 'mV'
    v_turb.long_name = 'Sea water turbidity (raw voltage)'
if fluo is not None:
    v_fluo = nc.createVariable('fluorescence', 'f4', ('obs',), zlib=True, fill_value=np.nan)
    v_fluo.units = 'mV'
    v_fluo.long_name = 'Fluorescence (raw voltage)'
if ph is not None:
    v_ph = nc.createVariable('pH', 'f4', ('obs',), zlib=True, fill_value=np.nan)
    v_ph.units = 'mV'
    v_ph.long_name = 'Sea water pH (raw voltage)'
if suna is not None:
    v_suna = nc.createVariable('suna_no3_voltage', 'f4', ('obs',), zlib=True, fill_value=np.nan)
    v_suna.units = 'mV'
    v_suna.long_name = 'SUNA nitrate channel (raw voltage)'

if lat is not None:
    v_lat = nc.createVariable('latitude', 'f4', ('obs',), zlib=True, fill_value=np.nan)
    v_lat.units = 'degrees_north'
    v_lat.standard_name = 'latitude'
if lon is not None:
    v_lon = nc.createVariable('longitude', 'f4', ('obs',), zlib=True, fill_value=np.nan)
    v_lon.units = 'degrees_east'
    v_lon.standard_name = 'longitude'
if time_concat is not None:
    v_time = nc.createVariable('time', 'f8', ('obs',), zlib=True, fill_value=np.nan)
    v_time.units = 'seconds since 1970-01-01T00:00:00Z'
    v_time.long_name = 'time'

# Write data
v_pres[:] = pres
v_temp[:] = temp if temp is not None else np.full(len(pres), np.nan)
v_sal[:]  = sal
v_do[:]   = do   if do   is not None else np.full(len(pres), np.nan)

if turb is not None: v_turb[:] = turb
if fluo is not None: v_fluo[:] = fluo
if ph   is not None: v_ph[:]   = ph
if suna is not None: v_suna[:] = suna

if lat is not None: v_lat[:] = lat
if lon is not None: v_lon[:] = lon
if time_concat is not None: v_time[:] = time_concat

# Global metadata
nc.title = 'MVP concatenated profiles'
nc.source = 'MVPAnalyzer export'
nc.history = f'Created on {datetime.utcnow().isoformat()}Z'
nc.Conventions = 'CF-1.8'
nc.profile_ids = ','.join(str(i) for i in profile_ids)

# Close file
nc.close()
print(f'NetCDF written to {outfile}')

In [7]:
mvpa.print_profile_metadata()

MVP data:
Number of profiles: 3
  Profil down 0 - Profil up 1 - Latitude: 42.89352, Longitude: 6.24231, Date/Heure: 2025-08-09 10:28:58.600000
  Profil down 2 - Profil up 3 - Latitude: 42.30967, Longitude: 6.09565, Date/Heure: 2025-08-11 00:01:16.800000
  Profil down 4 - Profil up 5 - Latitude: 42.47797, Longitude: 6.12631, Date/Heure: 2025-08-11 03:32:04.200000


In [3]:
mvpa.viscous_heating_correction()
print('Viscous heating correction done.')
mvpa.filtering_surface_waves()
print('Surface wave filtering done.')
mvpa.temporal_lag_correction(correction=True)
print('Temporal lag correction done.')

Viscous heating correction done.
Surface wave filtering done.
Temporal lag correction done.


In [ ]:
mvpa.thermal_mass_correction(save_param=True,sens_corr='down',max_depth=600)
# mvpa.thermal_mass_correction(load_param='param_slow.npy',sens_corr='down',max_depth=1000)
print('Thermal mass correction done.')

c:\Users\maxim\Documents\INGE CNRS\ESSTECH25\MVP_simple_routine\MVPAnalyzer\thermal_mass_correction.py:119: RuntimeWarning: invalid value encountered in divide
  Gamma_C = gamma_c/N


47.71206307411194  secondes pour le profil  0
41.34846091270447  secondes pour le profil  1
33.02995276451111  secondes pour le profil  2
66.27002739906311  secondes pour le profil  3
Thermal mass correction done.


In [5]:
%matplotlib tk
plt.figure()

id = 0
plt.plot(mvpa.SALT_align[id],mvpa.PRES0[id],label='MVP corrected down',color='black')
plt.plot(mvpa.SALT_align[id+1],mvpa.PRES0[id+1],label='MVP corrected up',color='red')
plt.plot(mvpa.SALT_final[id],mvpa.PRES0[id],label='MVP final down',color='blue')
plt.plot(mvpa.SALT_final[id+1],mvpa.PRES0[id+1],label='MVP final up',color='orange')
plt.gca().invert_yaxis()
plt.xlabel('Salinity (PSU)')
plt.ylabel('Pressure (dbar)')
plt.legend()
plt.title('Salinity profiles after corrections')
plt.show()

In [ ]:
mvpa.correct_oxygen([6],[6],plotting=False)

In [ ]:
mvpa.plot_TSprofile([0],correction=True)

In [ ]:
mvpa.stat_compar(cond=True)

In [ ]:
mvpa.plot_vertical_speed(4)

In [ ]:
mvpa.plot_profile_map()

In [ ]:
# %matplotlib tk
%matplotlib inline


mvpa.plot_BGCprofile(6) 

# mvpa.plot_BGCprofile(1)
# mvpa.plot_BGCprofile(2)

In [ ]:
# mvpa.plot_TSprofile(0)  
# mvpa.plot_TSprofile(1)  
%matplotlib tk
mvpa.plot_TSprofile(4)

In [ ]:
%matplotlib inline

In [ ]:

mvpa.plot_diagramTS_raw(0)
mvpa.plot_diagramTS_raw(2)
mvpa.plot_diagramTS_raw(4)  
mvpa.plot_diagramTS_raw(6)
mvpa.plot_diagramTS_raw()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure()
plt.plot(mvpa.PH_mvp[4],mvpa.PRES_mvp[4],label='MVP down')
plt.plot(mvpa.PH_mvp[5],mvpa.PRES_mvp[5],label='MVP up')
plt.gca().invert_yaxis()
plt.xlabel('pH')
plt.ylabel('Pressure, dbar')
plt.legend()
plt.grid()
plt.title('MVP pH profile')
plt.show()




In [ ]:

plt.figure()
plt.plot(mvpa.SUNA_mvp[5],mvpa.PRES_mvp[5],label='MVP down')
plt.plot(mvpa.SUNA_mvp[6],mvpa.PRES_mvp[6],label='MVP up')
plt.gca().invert_yaxis()
plt.xlabel('SUNA NO3 V')
plt.ylabel('Pressure, dbar')
plt.legend()
plt.grid()
plt.title('MVP NO3 profile')
plt.show()


In [30]:
plt.figure()
id = 4
plt.plot(mvpa.TEMP_mvp[id],mvpa.PRES_mvp[id],label='MVP down')
plt.plot(mvpa.TEMP_mvp[id+1],mvpa.PRES_mvp[id+1],label='MVP up')
plt.plot(mvpa.TEMP_ctd[id],mvpa.PRES_ctd[id],label='CTD down')
plt.plot(mvpa.TEMP_ctd[id+1],mvpa.PRES_ctd[id+1],label='CTD up')
plt.plot(mvpa.TEMP_mvp_corr[id],mvpa.PRES_mvp_corr[id],label='MVP corr down')
plt.plot(mvpa.TEMP_mvp_corr[id+1],mvpa.PRES_mvp_corr[id+1],label='MVP corr up')
plt.gca().invert_yaxis()
plt.xlabel('Temperature, °C')
plt.ylabel('Pressure, dbar')
plt.legend()
plt.grid()
plt.title('MVP Temperature Profiles')
plt.show()

import gsw
print(len(mvpa.COND_mvp_corr[id]), len(mvpa.TEMP_mvp_corr[id]), len(mvpa.PRES_mvp_corr[id]))
print(np.sum(np.isnan(mvpa.COND_mvp_corr[id])), np.sum(np.isnan(mvpa.TEMP_mvp_corr[id])), np.sum(np.isnan(mvpa.PRES_mvp_corr[id])))
newsalt = gsw.SP_from_C(mvpa.COND_mvp_corr[id], mvpa.TEMP_mvp_corr[id], mvpa.PRES_mvp_corr[id])

newsalt2 = gsw.SP_from_C(mvpa.COND_mvp_corr[id+1], mvpa.TEMP_mvp_corr[id+1], mvpa.PRES_mvp_corr[id+1])
plt.figure()
# plt.plot(mvpa.SALT_mvp[id],mvpa.PRES_mvp[id],label='MVP down')
# plt.plot(mvpa.SALT_mvp[id+1],mvpa.PRES_mvp[id+1],label='MVP up')
# plt.plot(mvpa.SALT_ctd[id],mvpa.PRES_ctd[id],label='CTD down')
# plt.plot(mvpa.SALT_ctd[id+1],mvpa.PRES_ctd[id+1],label='CTD up')
plt.plot(newsalt,mvpa.PRES_mvp_corr[id],label='MVP corr down')
plt.plot(newsalt2,mvpa.PRES_mvp_corr[id+1],label='MVP corr up')
plt.gca().invert_yaxis()
plt.xlabel('Salinity, psu')
plt.ylabel('Pressure, dbar')
plt.legend()
plt.grid()
plt.title('MVP Salinity Profiles')

28879 28879 28879
8621 28870 8621


Text(0.5, 1.0, 'MVP Salinity Profiles')

In [ ]:
%matplotlib tk
mvpa.plot_TSprofile(2,correction=True)

In [ ]:
%matplotlib inline

In [ ]:
mvpa.plot_correction(4)

In [ ]:
import gsw
import matplotlib.pyplot as plt
%matplotlib tk

id = 8

temp = mvpa.TEMP_mvp[id]
cond = mvpa.COND_mvp[id]
pres = mvpa.PRES_mvp[id]

mask = np.isnan(temp)==False
temp = temp[mask]
cond = cond[mask]
pres = pres[mask]


fs = mvpa.freq_echant  # Sampling frequency
v_z = np.gradient(pres, 1/fs)  # Vertical speed (dP/dt)
v_z = np.convolve(v_z, np.ones(41)/41, mode='same')  # Smooth vertical speed



from scipy.signal import butter, filtfilt

b,a = butter(3, 0.05 / 0.5 / 25 , btype='low')
temp_lowpass = filtfilt(b,a,temp)
cond_lowpass = filtfilt(b,a,cond)
pres_lowpass = filtfilt(b,a,pres)

sal_down = gsw.SP_from_C(cond, temp, pres)
sal_lowpass_down = gsw.SP_from_C(cond_lowpass, temp_lowpass, pres_lowpass)
sal_lowpass2_down = filtfilt(b,a,sal_down)

pres_down = pres


id = id+1

temp = mvpa.TEMP_mvp[id]
cond = mvpa.COND_mvp[id]
pres = mvpa.PRES_mvp[id]

mask = np.isnan(temp)==False
temp = temp[mask]
cond = cond[mask]
pres = pres[mask]


fs = mvpa.freq_echant  # Sampling frequency
v_z = np.gradient(pres, 1/fs)  # Vertical speed (dP/dt)
v_z = np.convolve(v_z, np.ones(41)/41, mode='same')  # Smooth vertical speed


b,a = butter(3, 0.2 / 0.5 / 25 , btype='low')
temp_lowpass = filtfilt(b,a,temp)
cond_lowpass = filtfilt(b,a,cond)
pres_lowpass = filtfilt(b,a,pres)

sal_up = gsw.SP_from_C(cond, temp, pres)
sal_lowpass_up = gsw.SP_from_C(cond_lowpass, temp_lowpass, pres_lowpass)
sal_lowpass2_up = filtfilt(b,a,sal_up)




plt.figure()
plt.plot(sal_lowpass_down, pres_down,label='Salinity down computed after filtration')
plt.plot(sal_lowpass_up, pres,label='Salinity up computed after filtration')
plt.plot(sal_down, pres_down, alpha=0.3,label='Salinity down unfiltered')
plt.plot(sal_up, pres, alpha=0.3,label='Salinity up unfiltered')
plt.legend()
plt.gca().invert_yaxis()
plt.xlabel('Salinity')
plt.ylabel('Pressure (dbar)')
plt.title('Low-pass filtered Salinity profile')
plt.grid()
plt.show()

check temoral lag between CTD and MVP

In [ ]:
id_mvp = 6
id_ctd = 6
import MVPAnalyzer.mvp_routines as mvp


temp = mvpa.TEMP_mvp[id_mvp]
cond = mvpa.COND_mvp[id_mvp]
pres = mvpa.PRES_mvp[id_mvp]

mask = np.isnan(temp)==False
temp = temp[mask]
cond = cond[mask]
pres = pres[mask]

temp_ctd = mvpa.TEMP_ctd[id_ctd]
cond_ctd = mvpa.COND_ctd[id_ctd]
pres_ctd = mvpa.PRES_ctd[id_ctd]
mask = np.isnan(temp_ctd)==False
temp_ctd = temp_ctd[mask]
cond_ctd = cond_ctd[mask]   
pres_ctd = pres_ctd[mask]

from scipy.signal import butter, filtfilt, correlate


b,a = butter(3, 10 / 0.5 / 25 , btype='low')
temp_lowpass = filtfilt(b,a,temp)
cond_lowpass = filtfilt(b,a,cond)
pres_lowpass = filtfilt(b,a,pres)

temp_highpass = temp - temp_lowpass
cond_highpass = cond - cond_lowpass
pres_highpass = pres - pres_lowpass

temp_ctd_lowpass = filtfilt(b,a,temp_ctd)
cond_ctd_lowpass = filtfilt(b,a,cond_ctd)
pres_ctd_lowpass = filtfilt(b,a,pres_ctd)

temp_ctd_highpass = temp_ctd - temp_ctd_lowpass
cond_ctd_highpass = cond_ctd - cond_ctd_lowpass
pres_ctd_highpass = pres_ctd - pres_ctd_lowpass

l_lag = np.arange(-10,11,0.01)
l_corr = []

num_sample = 3000
pmin = np.nanmin(mvpa.PRES_mvp)
pmax = np.nanmax(mvpa.PRES_mvp)
pressure_grid = np.linspace(pmin, pmax, num_sample)

temp_ctd_interp = mvp.vertical_interp(pres_ctd.reshape(1, -1), temp_ctd_lowpass.reshape(1, -1), pressure_grid)
temp_interp = mvp.vertical_interp(pres.reshape(1, -1), temp_lowpass.reshape(1, -1), pressure_grid)

corr = correlate(temp_interp,temp_ctd_interp,mode='full')
lags = np.arange(-len(temp_interp)+1,len(temp_ctd_interp))

lag_opt = lags[np.argmax(corr)]
dp = abs(pressure_grid[0]-pressure_grid[1])  # pas de temps
time_lag = lag_opt * dt



print(f"Décalage optimal : {lag_opt} points, soit environ {time_lag:.2f} mètres")

import matplotlib.pyplot as plt


plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(pressure_grid, temp_interp.flatten(), label="Capteur 1")
plt.plot(pressure_grid, temp_ctd_interp.flatten(), label="Capteur 2 (décalé)")
plt.legend()
plt.title("Signaux bruts")

plt.subplot(1,2,2)
plt.plot(lags*dt, corr)
plt.axvline(time_lag, color='r', linestyle='--', label=f"Lag = {time_lag:.2f}s")
plt.title("Corrélation croisée")
plt.xlabel("Décalage (s)")
plt.legend()
plt.show()



In [ ]:
id_mvp = 0
id_ctd = 0
import MVPAnalyzer.mvp_routines as mvp


temp = mvpa.TEMP_mvp[id_mvp]
cond = mvpa.COND_mvp[id_mvp]
pres = mvpa.PRES_mvp[id_mvp]

mask = np.isnan(temp)==False
temp = temp[mask]
cond = cond[mask]
pres = pres[mask]

temp_ctd = mvpa.TEMP_ctd[id_ctd]
cond_ctd = mvpa.COND_ctd[id_ctd]
pres_ctd = mvpa.PRES_ctd[id_ctd]
mask = np.isnan(temp_ctd)==False
temp_ctd = temp_ctd[mask]
cond_ctd = cond_ctd[mask]   
pres_ctd = pres_ctd[mask]

from scipy.signal import butter, filtfilt, correlate


b,a = butter(3, 10 / 0.5 / 25 , btype='low')
temp_lowpass = filtfilt(b,a,temp)
cond_lowpass = filtfilt(b,a,cond)
pres_lowpass = filtfilt(b,a,pres)

temp_highpass = temp - temp_lowpass
cond_highpass = cond - cond_lowpass
pres_highpass = pres - pres_lowpass

temp_ctd_lowpass = filtfilt(b,a,temp_ctd)
cond_ctd_lowpass = filtfilt(b,a,cond_ctd)
pres_ctd_lowpass = filtfilt(b,a,pres_ctd)

temp_ctd_highpass = temp_ctd - temp_ctd_lowpass
cond_ctd_highpass = cond_ctd - cond_ctd_lowpass
pres_ctd_highpass = pres_ctd - pres_ctd_lowpass

l_lag = np.arange(-10,11,0.01)
l_corr = []
l_corr2 = []

num_sample = 3000
pmin = np.nanmin(pres_ctd)
pmax = np.nanmax(pres_ctd)
pressure_grid = np.linspace(pmin, pmax, num_sample)

# temp_ctd_interp = np.interp(pressure_grid,pres_ctd, temp_ctd_lowpass)
temp_ctd_interp = mvp.vertical_interp(pres_ctd.reshape(1,-1),temp_ctd_lowpass.reshape(1,-1),pressure_grid)
cond_ctd_interp = mvp.vertical_interp(pres_ctd.reshape(1,-1),cond_ctd_lowpass.reshape(1,-1),pressure_grid)

# print(np.sum(np.isnan(pres_ctd)))
# print(np.all(np.diff(pres_ctd) > 0))  # True si croissant strict
# print(np.sum(np.isnan(temp_ctd_interp)))

for lag in l_lag:
    pres_lag = pres + lag
    # temp_interp = np.interp(pressure_grid,pres, temp_lowpass)
    temp_interp =  mvp.vertical_interp(pres_lag.reshape(1,-1),temp_lowpass.reshape(1,-1),pressure_grid)
    mask = ~np.isnan(temp_interp) & ~np.isnan(temp_ctd_interp)
    temp_interp = temp_interp[mask]
    temp_ctd_interp2 = temp_ctd_interp[mask]
    corr = np.corrcoef(temp_interp,temp_ctd_interp2)[0,1]
    l_corr.append(corr)


corr_max = np.nanmax(l_corr)
lag_max = l_lag[np.nanargmax(l_corr)]
print(f"Correlation maximale : {corr_max:.4f} au lag de {lag_max:.2f} dbar")

import matplotlib.pyplot as plt


for lag in l_lag:
    pres_lag = pres + lag
    # temp_interp = np.interp(pressure_grid,pres, temp_lowpass)
    cond_interp =  mvp.vertical_interp(pres_lag.reshape(1,-1),cond_lowpass.reshape(1,-1),pressure_grid)
    mask = ~np.isnan(cond_interp) & ~np.isnan(cond_ctd_interp)
    cond_interp = cond_interp[mask]
    cond_ctd_interp2 = cond_ctd_interp[mask]
    corr = np.corrcoef(cond_interp,cond_ctd_interp2)[0,1]
    l_corr2.append(corr)


corr_max2 = np.nanmax(l_corr2)
lag_max2 = l_lag[np.nanargmax(l_corr2)]
print(f"Correlation maximale : {corr_max2:.4f} au lag de {lag_max2:.2f} dbar")

import matplotlib.pyplot as plt


plt.figure()

plt.plot(l_lag, l_corr,label='Temperature')
plt.plot(l_lag, l_corr2,label='Conductivity')
plt.axvline(lag_max, color='r', linestyle='--', label=f"Lag = {lag_max:.2f}dbar")
plt.axvline(lag_max2, color='g', linestyle='--', label=f"Lag = {lag_max2:.2f}dbar")
plt.title("Corrélation croisée")
plt.legend()
plt.xlabel("Décalage (dbar)")
plt.legend()
plt.show()



In [ ]:
def find_lag(l_mvp,l_ctd,pres_mvp,pres_ctd,lowpass=False,highpass=False):

    mask = np.isnan(l_mvp)==False
    l_mvp = l_mvp[mask]
    pres_mvp = pres_mvp[mask]

    mask = np.isnan(l_ctd)==False
    l_ctd = l_ctd[mask]
    pres_ctd = pres_ctd[mask]

    if lowpass:
        from scipy.signal import butter, filtfilt
        b,a = butter(3, 0.1 / 0.5 / 25 , btype='low')
        l_mvp = filtfilt(b,a,l_mvp)
        l_ctd = filtfilt(b,a,l_ctd)
    if highpass:
        from scipy.signal import butter, filtfilt
        b,a = butter(3, 10 / 0.5 / 25 , btype='high')
        l_mvp = filtfilt(b,a,l_mvp)
        l_ctd = filtfilt(b,a,l_ctd)
    
    l_lag = np.arange(-10,11,0.01)
    l_corr = []

    num_sample = 3000
    pmin = np.nanmin(pres_ctd)
    pmax = np.nanmax(pres_ctd)
    pressure_grid = np.linspace(pmin, 100, num_sample)

    # temp_ctd_interp = np.interp(pressure_grid,pres_ctd, temp_ctd_lowpass)
    import MVPAnalyzer.mvp_routines as mvp
    l_ctd_interp = mvp.vertical_interp(pres_ctd.reshape(1,-1),l_ctd.reshape(1,-1),pressure_grid)

    # print(np.sum(np.isnan(pres_ctd)))
    # print(np.all(np.diff(pres_ctd) > 0))  # True si croissant strict
    # print(np.sum(np.isnan(temp_ctd_interp)))

    for lag in l_lag:
        pres_lag = pres_mvp + lag
        # temp_interp = np.interp(pressure_grid,pres, temp_lowpass)
        l_interp =  mvp.vertical_interp(pres_lag.reshape(1,-1),l_mvp.reshape(1,-1),pressure_grid)
        mask = ~np.isnan(l_interp) & ~np.isnan(l_ctd_interp)
        l_interp = l_interp[mask]
        l_ctd_interp2 = l_ctd_interp[mask]
        corr = np.corrcoef(l_interp,l_ctd_interp2)[0,1]
        l_corr.append(corr)


    corr_max = np.nanmax(l_corr)
    lag_max = l_lag[np.nanargmax(l_corr)]
    print(f"Correlation maximale : {corr_max:.4f} au lag de {lag_max:.2f} dbar")
    return lag_max, corr_max, l_lag, l_corr
    

In [ ]:
from scipy.signal import butter, filtfilt, correlate
def find_lag2(mvp,p_mvp, ctd,p_ctd, fs):
    # Interpoler sur une grille de pression commune (liée à p_mvp et p_ctd)
    mvp = np.asarray(mvp).ravel()
    p_mvp = np.asarray(p_mvp).ravel()
    ctd = np.asarray(ctd).ravel()
    p_ctd = np.asarray(p_ctd).ravel()

    # Nettoyage NaN
    mask_mvp = (~np.isnan(mvp)) & (~np.isnan(p_mvp))
    mask_ctd = (~np.isnan(ctd)) & (~np.isnan(p_ctd))
    mvp = mvp[mask_mvp]; p_mvp = p_mvp[mask_mvp]
    ctd = ctd[mask_ctd]; p_ctd = p_ctd[mask_ctd]

    # Trier par pression croissante
    im = np.argsort(p_mvp); ic = np.argsort(p_ctd)
    p_mvp_s, mvp_s = p_mvp[im], mvp[im]
    p_ctd_s, ctd_s = p_ctd[ic], ctd[ic]

    # Emprise commune
    pmin_c = max(np.nanmin(p_mvp_s), np.nanmin(p_ctd_s))
    pmax_c = min(np.nanmax(p_mvp_s), np.nanmax(p_ctd_s))
    if not np.isfinite(pmin_c) or not np.isfinite(pmax_c) or pmax_c <= pmin_c:
        return 0.0, np.array([]), np.array([])

    # Grille de pression commune
    n_grid = 3000
    p_grid = np.linspace(pmin_c,pmax_c, n_grid)

    # Interpolation sans extrapolation
    mvp_i = np.interp(p_grid, p_mvp_s, mvp_s, left=np.nan, right=np.nan)
    ctd_i = np.interp(p_grid, p_ctd_s, ctd_s, left=np.nan, right=np.nan)

    # Points valides communs
    m = (~np.isnan(mvp_i)) & (~np.isnan(ctd_i))
    mvp = mvp_i[m]
    ctd = ctd_i[m]

    # enlever tendance / normaliser
    mvpn = (mvp - np.mean(mvp))
    ctdn = (ctd - np.mean(ctd))

    corr = correlate(mvpn, ctdn, mode='full')
    lags = np.arange(-len(mvpn)+1, len(ctdn))
    lag_idx = lags[np.argmax(np.abs(corr))]
    lag_seconds = lag_idx / fs
    
    return lag_seconds,corr,lags

In [ ]:
id = 2
lag,corr,lags = find_lag2(mvpa.TEMP_mvp[id], mvpa.PRES_mvp[id], mvpa.TEMP_ctd[id], mvpa.PRES_ctd[id], mvpa.freq_echant)

print(f"Décalage optimal : {lag} secondes")
plt.figure()
plt.plot(lags,corr)
plt.axvline(lag*mvpa.freq_echant, color='r', linestyle='--', label=f"Lag = {lag:.2f}s") 
plt.title("Corrélation croisée")
plt.xlabel("Décalage (échantillons)")
plt.ylabel("Corrélation")
plt.legend()
plt.show()

In [ ]:
lag_max,corr_max,l_lag,l_cor = find_lag(mvpa.TEMP_mvp[i],mvpa.TEMP_ctd[i],mvpa.PRES_mvp[i],mvpa.PRES_ctd[i],lowpass=True)

print(f"Décalage optimal : {lag_max} dbar")
plt.figure()
plt.plot(l_lag,l_cor)
plt.axvline(lag_max, color='r', linestyle='--', label=f"Lag = {lag_max:.2f}dbar") 
plt.title("Corrélation croisée")
plt.xlabel("Décalage (dbar)")
plt.ylabel("Corrélation")
plt.legend()
plt.show()

In [ ]:
from scipy.signal import butter, filtfilt
lag_t = []
lag_c= []

for i in range(0,len(mvpa.PRES_mvp)):
    
    lag_t.append(find_lag(mvpa.TEMP_mvp[i],mvpa.TEMP_ctd[i],mvpa.PRES_mvp[i],mvpa.PRES_ctd[i],lowpass=True)[0])
    lag_c.append(find_lag(mvpa.COND_mvp[i],mvpa.COND_ctd[i],mvpa.PRES_mvp[i],mvpa.PRES_ctd[i],lowpass=True)[0])


plt.figure()
plt.plot(lag_t,label='temp')
plt.plot(lag_c,label='cond')
plt.legend()
plt.show()

In [ ]:

lag_tc = []
for i in range(0,len(mvpa.PRES_mvp)):
    lag_tc.append(find_lag(mvpa.COND_mvp[i],mvpa.TEMP_mvp[i],mvpa.PRES_mvp[i],mvpa.PRES_mvp[i],lowpass=False,highpass=True)[0])


plt.figure()

plt.plot(lag_tc,label='cond/temp')
plt.legend()
plt.show()

test lag correction


In [ ]:


id = 6

pres = mvpa.PRES_mvp[id]
temp = mvpa.TEMP_mvp[id]
cond = mvpa.COND_mvp[id]

mask = np.isnan(temp)==False
temp = temp[mask]
cond = cond[mask]
pres = pres[mask]

pres_ctd = mvpa.PRES_ctd[id]
temp_ctd = mvpa.TEMP_ctd[id]
cond_ctd = mvpa.COND_ctd[id]

mask = np.isnan(temp_ctd)==False
temp_ctd = temp_ctd[mask]
cond_ctd = cond_ctd[mask]
pres_ctd = pres_ctd[mask]

from scipy.signal import butter, filtfilt
b,a = butter(3, 0.1 / 0.5 / 25 , btype='low')
temp_filt = filtfilt(b,a,temp)
cond_filt = filtfilt(b,a,cond)



num_sample = 3000
pmin = np.nanmin(pres_ctd)
pmax = np.nanmax(pres_ctd)
pressure_grid = np.linspace(pmin, pmax, num_sample)


import MVPAnalyzer.mvp_routines as mvp

temp_interp = mvp.vertical_interp(pres.reshape(1,-1),temp.reshape(1,-1),pressure_grid)
temp_ctd_interp = mvp.vertical_interp(pres_ctd.reshape(1,-1),temp_ctd.reshape(1,-1),pressure_grid)
temp_interp_corr = mvp.vertical_interp(pres.reshape(1,-1)+lag_t[id],temp.reshape(1,-1),pressure_grid)
temp_interp_corr2 = mvp.vertical_interp(pres.reshape(1,-1)+lag_t[id],temp_filt.reshape(1,-1),pressure_grid)

cond_interp = mvp.vertical_interp(pres.reshape(1,-1),cond.reshape(1,-1),pressure_grid)
cond_ctd_interp = mvp.vertical_interp(pres_ctd.reshape(1,-1),cond_ctd.reshape(1,-1),pressure_grid)
cond_interp_corr = mvp.vertical_interp(pres.reshape(1,-1)+lag_c[id],cond.reshape(1,-1),pressure_grid)
cond_interp_corr2 = mvp.vertical_interp(pres.reshape(1,-1)+lag_c[id],cond_filt.reshape(1,-1),pressure_grid)

mask1 = ~np.isnan(temp_interp) & ~np.isnan(cond_interp) 
print(temp_interp.shape, cond_interp.shape, pressure_grid.shape,mask1.shape)
temp_interp = temp_interp[mask1]
cond_interp = cond_interp[mask1]
pressure_grid1 = pressure_grid[mask1[0]]

mask2 = ~np.isnan(temp_interp_corr) & ~np.isnan(cond_interp_corr)
temp_interp_corr = temp_interp_corr[mask2]
cond_interp_corr = cond_interp_corr[mask2]
pressure_grid2 = pressure_grid[mask2[0]]

mask3 = ~np.isnan(temp_ctd_interp) & ~np.isnan(cond_ctd_interp)
temp_ctd_interp = temp_ctd_interp[mask3]
cond_ctd_interp = cond_ctd_interp[mask3]
pressure_grid3 = pressure_grid[mask3[0]]

mask4 = ~np.isnan(temp_interp_corr2) & ~np.isnan(cond_interp_corr2)
temp_interp_corr2 = temp_interp_corr2[mask4]
cond_interp_corr2 = cond_interp_corr2[mask4]
pressure_grid4 = pressure_grid[mask4[0]]

import gsw


sal = gsw.SP_from_C(cond_interp.flatten(), temp_interp.flatten(), pressure_grid1)
sal_corr = gsw.SP_from_C(cond_interp_corr.flatten(), temp_interp_corr.flatten(), pressure_grid2)
sal_ctd = gsw.SP_from_C(cond_ctd_interp.flatten(), temp_ctd_interp.flatten(), pressure_grid3)
sal_filt = gsw.SP_from_C(cond_filt.flatten(), temp_filt.flatten(), pres.flatten())
sal_corr2 = gsw.SP_from_C(cond_interp_corr2.flatten(), temp_interp_corr2.flatten(), pressure_grid4)

# rmse_before_t = np.sqrt(np.nanmean((temp_interp - temp_ctd_interp)**2))
# rmse_after_t = np.sqrt(np.nanmean((temp_interp_corr - temp_ctd_interp)**2))
# rmse_before_c = np.sqrt(np.nanmean((cond_interp - cond_ctd_interp)**2))
# rmse_after_c = np.sqrt(np.nanmean((cond_interp_corr - cond_ctd_interp)**2))
# rmse_before_s = np.sqrt(np.nanmean((sal - sal_ctd)**2))
# rmse_after_s = np.sqrt(np.nanmean((sal_corr - sal_ctd)**2))

# print(f"Temperature RMSE before correction: {rmse_before_t:.4f}")
# print(f"Temperature RMSE after correction: {rmse_after_t:.4f}")
# print(f"Conductivity RMSE before correction: {rmse_before_c:.4f}")
# print(f"Conductivity RMSE after correction: {rmse_after_c:.4f}")
# print(f"Salinity RMSE before correction: {rmse_before_s:.4f}")
# print(f"Salinity RMSE after correction: {rmse_after_s:.4f}")



import matplotlib.pyplot as plt
%matplotlib tk
plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plt.plot(temp_interp.flatten(), pressure_grid1, label='MVP')
plt.plot(temp_ctd_interp.flatten(), pressure_grid3, label='CTD')
plt.plot(temp_interp_corr.flatten(), pressure_grid2, label='MVP corr')
plt.gca().invert_yaxis()
plt.xlabel('Temperature (°C)')
plt.ylabel('Pressure (dbar)')
plt.title('Temperature Profile')
plt.legend()
plt.subplot(1,3,2)
plt.plot(cond_interp.flatten(), pressure_grid1, label='MVP')
plt.plot(cond_ctd_interp.flatten(), pressure_grid3, label='CTD')
plt.plot(cond_interp_corr.flatten(), pressure_grid2, label='MVP corr')
plt.gca().invert_yaxis()
plt.xlabel('Conductivity (mS/cm)')
plt.title('Conductivity Profile')
plt.legend()
plt.subplot(1,3,3)
plt.plot(sal, pressure_grid1.flatten(), label='MVP')
plt.plot(sal_ctd, pressure_grid3.flatten(), label='CTD')
plt.plot(sal_corr, pressure_grid2.flatten(), label='MVP corr')
plt.plot(sal_filt, pres, label='MVP filt')
plt.plot(sal_corr2, pressure_grid4.flatten(), label='MVP corr2')
plt.gca().invert_yaxis()
plt.xlabel('Salinity (PSU)')
plt.title('Salinity Profile')
plt.legend()
plt.tight_layout()
plt.show()


test treatment routines

In [ ]:


## Filtering high frequencies

from scipy.signal import butter, filtfilt
fc = 4 / 25  # Cut-off frequency (Hz) for a 4-scan period at 25 Hz sampling
b,a = butter(3, fc, btype='low')

Pres_filt = np.zeros_like(mvpa.PRES_mvp)
Temp_filt = np.zeros_like(mvpa.TEMP_mvp)
Cond_filt = np.zeros_like(mvpa.COND_mvp)

for i in range(0,len(mvpa.PRES_mvp)):
    Pres_filt[i] = filtfilt(b,a,mvpa.PRES_mvp[i])
    Temp_filt[i] = filtfilt(b,a,mvpa.TEMP_mvp[i])
    Cond_filt[i] = filtfilt(b,a,mvpa.COND_mvp[i])


## 

    